In [2]:
import gc
import os
import json

import pandas as pd
import pyarrow.parquet as pq
import sqlite3
from sqlite3 import Error


In [3]:
config = {
    'data_dir': os.path.join('data'), 
    'data_db_storage_dir': os.path.join('data', 'db_storage'),
    
    'data_db_name': 'yelp_data',
    'data_db_access_uname': 'db_admin_1', 
    'data_db_access_pwd': 'pass1234',
}

In [5]:
class DatabaseDMLManager:
    '''
    A class that manages a SQLite database connection and provides methods for interacting with the database.
    
    Attributes:
        db_name (str): The name of the database.
        conn (sqlite3.Connection): The SQLite database connection.
    '''
    def __init__(self, db_name):
        self.db_name = db_name
        # self.conn = self.open_connection()

    def open_connection(self):
        '''
        Opens a connection to the database.
        
        Returns:
            sqlite3.Connection: The SQLite database connection; 
                Or None if failed.
        '''
        try:
            self.conn = sqlite3.connect(self.db_name)
            return self.conn
        except Error as e:
            print(e)
        
        return None
    
    def close_connection(self):
        '''
        Closes the database connection.
        '''
        self.conn.close()
    
    def get_table_names(self):
        '''
        Retrieves the names of all tables in the database.

        Returns:
            list: A list of table names.
        '''
        table_names = []
        try:
            with sqlite3.connect(self.db_name) as conn:
                cursor = conn.cursor()
                cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
                tables = cursor.fetchall()

                table_names = [table[0] for table in tables]
        except Error as e:
            print(e)
            
        return table_names
    
    def execute_query(self, query_str):
        try:
            with sqlite3.connect(self.db_name) as conn:
                cursor = conn.cursor()
                cursor.execute(query_str)
                result = cursor.fetchall()
                column_names = [desc[0] for desc in cursor.description]
                return [column_names] + list(result)
        except Error as e:
            print("Error Message: ", e)
        
        return None
    
    def get_table_columns(self, table_name):
        '''
        Retrieves the column names of a specific table.
        
        Args:
            table_name (str): The name of the table.
        
        Returns:
            list: A list of column names.
        '''
        try:
            with sqlite3.connect(self.db_name) as conn:
                cursor = conn.cursor()
                cursor.execute("PRAGMA table_info({})".format(table_name))
                columns = [column[1] for column in cursor.fetchall()]
                return columns
        except Error as e:
            print(e)
            
        return None
    
    def get_table_number_of_rows(self, table_name):
        '''
        Retrieves the number of rows in a specific table.
        
        Args:
            table_name (str): The name of the table.
        
        Returns:
            int: The number of rows in the table.
        '''
        try:
            with sqlite3.connect(self.db_name) as conn:
                cursor = conn.cursor()
                cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
                count = cursor.fetchone()
                return count[0]
        except Error as e:
            print(e)
        
        return None
    
    def fetch_column_from_table(self, table_name, column_name, sample_size=None):
        '''
        Retrieves a specific column from a table.
        
        Args:
            table_name (str): The name of the table.
            column_name (str): The name of the column.
            sample_size (int, optional): The number of rows to retrieve. Defaults to None.
        
        Returns:
            list: A list of values from the specified column.
        '''
        assert sample_size is None or (isinstance(sample_size, int) and sample_size > 0), \
                    "Sample size must be a positive integer."
        
        assert table_name in ['business', 'checkin', 'review', 'tip', 'user'], \
                    "Invalid table name."
        
        assert column_name in self.get_table_columns(table_name), \
                "Invalid column name. {} does not exist in the table {}.".\
                format(column_name, table_name)
        
        result = None
        try:
            with sqlite3.connect(self.db_name) as conn:
                cursor = conn.cursor()
                if sample_size:
                    cursor.execute(f"SELECT {column_name} FROM {table_name} \
                            LIMIT {sample_size}")
                    result = cursor.fetchmany(sample_size)
                else:
                    cursor.execute(f"SELECT {column_name} FROM {table_name}")
                    result = cursor.fetchall()
        except Error as e:
            print(e)
        
        return result

    def read_db_table(self, table_name, sample_size=None, verbose=False):
        '''
        Retrieves all/requested number of rows from a specific table.
        
        Args:
            table_name (str): The name of the table.
            sample_size (int, optional): The number of rows to retrieve. Defaults to None.
            verbose (bool, optional): Whether to print the retrieved rows. Defaults to False.
        
        Returns:
            list: A list of rows from the specified table.
        '''
        assert sample_size is None or (isinstance(sample_size, int) and sample_size > 0), \
                    "Sample size must be a positive integer."   

        assert table_name in ['business', 'checkin', 'review', 'tip', 'user'], \
                    "Invalid table name."
        
        result = None
        try:
            with sqlite3.connect(self.db_name) as conn:
                cursor = conn.cursor()
                if sample_size:
                    cursor.execute(f"SELECT * FROM {table_name} LIMIT \
                            {sample_size}".format(table_name))
                    result = cursor.fetchmany(sample_size)
                else:
                    cursor.execute(f"SELECT * FROM {table_name}".format(table_name))
                    result = cursor.fetchall()
                
                if sample_size and verbose:
                    for row in result:
                        print(row)
        except Error as e:
            print(e)
            
        return result
    
    def insert_data(self, table_name, data_dict):
        '''
        Inserts data into the database.
        Leaving to SQL to generate errors for invalid values like that 
            not matching the data type or primary key or null values in non-null 
            columns or duplicate values in unique columns.
        
        Args:
            table_name (str): The name of the table.
            data_dict (dict): The data to insert.

        Returns:
            Boolean: True if the data was inserted successfully, False otherwise.        
        '''
        try:
            with sqlite3.connect(self.db_name) as conn:
                cursor = conn.cursor()
                columns = ', '.join(data_dict.keys())

                # Check if all columns in data_dict are valid columns in the table
                valid_columns = self.get_table_columns(table_name)
                invalid_columns = [column for column in data_dict.keys() \
                                   if column not in valid_columns]
                if invalid_columns:
                    print(f"Invalid columns: {', '.join(invalid_columns)}")
                    return False

                # Create a string of placeholders for the values to insert
                # in order to prevent SQL injection attacks
                placeholders = ', '.join(['?' for _ in range(len(data_dict))])
                values = tuple(data_dict.values())
                try:
                    cursor.execute(f"INSERT INTO {table_name} ({columns}) VALUES \
                                ({placeholders})", values)
                    self.conn.commit()
                    return True
                except Exception as e:
                    # For table constraints and other value errors
                    print(f"Error inserting data into {table_name}: {e}")
                    return False
        except Error as e:
            print(e)
            
        return False

    def update_data(self, table_name, data_dict):
        '''
        Updates data in the database.

        Args:
            table_name (str): The name of the table.
            data_dict (dict): The new data to update.

        Returns:
            Boolean: True if the data was updated successfully, False otherwise.
        '''
        try:
            with sqlite3.connect(self.db_name) as conn:
                cursor = conn.cursor()

                # Check if all columns in data_dict are valid columns in the table
                valid_columns = self.get_table_columns(table_name)
                invalid_columns = [column for column in data_dict.keys() if column not in valid_columns]
                if invalid_columns:
                    print(f"Invalid columns: {', '.join(invalid_columns)}")
                    return False

                # Create a string of placeholders for the values to update
                # in order to prevent SQL injection attacks
                placeholders = ', '.join([f"{column} = ?" for column in data_dict.keys()])
                values = tuple(data_dict.values())
                try:
                    cursor.execute(f"UPDATE {table_name} SET {placeholders}", values)
                    self.conn.commit()
                    return True
                except Exception as e:
                    print(f"Error updating data in {table_name}: {e}")
                    return False
        except Error as e:
            print(e)
        
        return False
        
    def __get_primary_key_columns(self, table_name):
        '''
        Retrieves the primary key column(s) of a table.

        Args:
            table_name (str): The name of the table.

        Returns:
            list: A list of primary key column(s).
        '''
        try:
            with sqlite3.connect(self.db_name) as conn:
                cursor = self.conn.cursor()
                cursor.execute(f"PRAGMA table_info({table_name})")
                table_info = cursor.fetchall()

                # table_info structure in sqlite looks like:
                # (column_id, column_name, data_type, not_null, default_value, is_primary_key)

                # is_primary_key is 1 for all the columns in the composite primary key
                primary_key_columns = [column[1] for column in table_info if column[5] == 1]

                return primary_key_columns
        except Error as e:
            print(e)
        
        return None
            
    def __get_deletion_columns(self, table_name, deletion_dict):
        '''
        Retrieves the columns to use as the deletion key(s) from the deletion_dict.
        '''
        table_prim_key_columns = self.__get_primary_key_columns(table_name)
        if not table_prim_key_columns:
            # IMPORTANT NOTE: Ideally, we should not allow deletion without a primary key
            # But allowing it for now
            deleteion_columns = list(deletion_dict.keys())
        else:
            # check if the deletion key(s) are keys of the table - primary or composite etc.
            # If passed multiple columns; Filter out only the columns that form the 
            # primary key
            deleteion_columns = []
            for column in deletion_dict.keys():
                if column not in table_prim_key_columns:
                    print(f'Column {column} is not a primary key column.' \
                        'Cannot delete with this key.')
                    return False, None
                else: 
                    deleteion_columns.append(column)

            # ensure all columns that form the primary key are passed in the deletion_dict
            # to prevent accidental deletion of multiple rows in case of composite primary key
            if len(deleteion_columns) != len(table_prim_key_columns):
                print(f"Not all primary key columns are not passed in the deletion_dict.")
                print("Key deleteion_columns: ", deleteion_columns)
                print("table_prim_key_columns: ", table_prim_key_columns)
                return False, None

        return deleteion_columns
        
    def delete_data(self, table_name, deletion_dict, return_deleted_data=False):
        '''
            Deletes data from the database based on the provided key-value 
            pairs in the deletion_dict.

        Args:
            table_name (str): The name of the table.
            deletion_dict (dict): A dictionary containing the key-value pairs 
                        to use as the deletion key-value pairs.
            return_deleted_data (bool, optional): Whether to return the deleted 
                    data. Defaults to False.

        Returns:
            tuple: A tuple containing a boolean flag indicating the success of 
                    the operation and the deleted data 
                    (if return_deleted_data is True).
        '''
        try:
            with sqlite3.connect(self.db_name) as conn:
                cursor = conn.cursor()

                # Check if deletion_key_name is a valid column in the table
                valid_columns = self.get_table_columns(table_name)
                for column in deletion_dict.keys():
                    if column not in valid_columns:
                        print(f"Invalid column: {column}")
                        return False, None

                deleteion_columns = self.__get_deletion_columns(table_name, deletion_dict)

                # Create a string of placeholders for the WHERE clause
                placeholders = ' AND '.join([f"{column} = ?" for column in deleteion_columns])
                values = tuple(deletion_dict.values())

                # Check if the column can be deleted based on table constraints
                # Add your own logic here to check the constraints
                try:
                    # Select the data to be returned before deletion
                    if return_deleted_data:
                        cursor.execute(f"SELECT * FROM {table_name} WHERE \
                                       {placeholders}", values)
                        deleted_data = cursor.fetchall()
                    else:
                        deleted_data = None

                    # Delete the data based on the deletion key
                    cursor.execute(f"DELETE FROM {table_name} WHERE  {placeholders}", \
                                   values)
                    self.conn.commit()

                    return True, deleted_data
                except Exception as e:
                    print(f"Error deleting data from {table_name}: {e}")
                    return False, None
        except Error as e:
            print(e)

    def read_data(self, table_name, key_val_dict):
        '''
        Reads data from the database based on the provided key-value pairs.
        Can handle multiple key-value pairs.

        Args:
            table_name (str): The name of the table.
            key_val_dict (dict): A dictionary containing the key-value pairs 
                    to use for reading the data.

        Returns:
            list: A list of rows from the specified table.
        '''
        try:
            with sqlite3.connect(self.db_name) as conn:
                cursor = self.conn.cursor()

                # Create a string of placeholders for the WHERE clause
                placeholders = ' AND '.join([f"{column} = ?" for column in \
                                             key_val_dict.keys()])
                values = tuple(key_val_dict.values())

                try:
                    cursor.execute(f"SELECT * FROM {table_name} WHERE {placeholders}", \
                                   values)
                    result = cursor.fetchall()
                    return result
                except Exception as e:
                    print(f"Error reading data from {table_name}: {e}")
                    return [] 
        except Error as e:
            print(e)
        
        return None
        


In [14]:
db_path = os.path.join(config['data_db_storage_dir'], config['data_db_name'])
dml_man = DatabaseDMLManager(db_path)


In [15]:
table_names = dml_man.get_table_names()
table_names

['business', 'user', 'review', 'tip', 'checkin']

In [16]:
for tname in table_names:
    cols = dml_man.get_table_columns(tname)
    print("Table: ", tname)
    print("Columns: ", cols)
    print("-"*10)

Table:  business
Columns:  ['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'categories']
----------
Table:  user
Columns:  ['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny', 'cool', 'fans', 'average_stars', 'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_funny', 'compliment_writer', 'compliment_photos']
----------
Table:  review
Columns:  ['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date']
----------
Table:  tip
Columns:  ['user_id', 'business_id', 'text', 'date', 'compliment_count']
----------
Table:  checkin
Columns:  ['business_id', 'date']
----------


In [17]:
for tname in table_names:
    cols = dml_man.get_table_number_of_rows(tname)
    print("Table: ", tname)
    print("# Rows: ", cols)
    print("-"*10)



Table:  business
# Rows:  150346
----------
Table:  user
# Rows:  1987897
----------
Table:  review
# Rows:  6990280
----------
Table:  tip
# Rows:  908915
----------
Table:  checkin
# Rows:  131930
----------


In [18]:

for tname in ['business']:
    cols = dml_man.fetch_column_from_table(tname, 'name', sample_size=10)
    print("Table: ", tname)
    print("# Rows: ", cols)
    print("-"*10)


Table:  business
# Rows:  [('Abby Rappoport, LAC, CMQ',), ('The UPS Store',), ('Target',), ('St Honore Pastries',), ('Perkiomen Valley Brewery',), ('Sonic Drive-In',), ('Famous Footwear',), ('Temple Beth-El',), ("Tsevi's Pub And Grill",), ('Sonic Drive-In',)]
----------


In [19]:
table_data = dml_man.read_db_table('business', sample_size=2, verbose=False)
table_data

[('Pns2l4eNsfO8kk83dixA6A',
  'Abby Rappoport, LAC, CMQ',
  '1616 Chapala St, Ste 2',
  'Santa Barbara',
  'CA',
  '93101',
  34.42667770385742,
  -119.71119689941406,
  5,
  7,
  0,
  'Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists'),
 ('mpf3x-BjTdTEA3yCZrAYPw',
  'The UPS Store',
  '87 Grasso Plaza Shopping Center',
  'Affton',
  'MO',
  '63123',
  38.551124572753906,
  -90.335693359375,
  3,
  15,
  1,
  'Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services')]

# 

## How many distinct cities do we have in the dataset?

In [20]:
query_str = '''select count(distinct(city)) as 'Cities Count' from business limit 10'''
result = dml_man.execute_query(query_str)

# print("Number of distinct cities : ", result[0][0])

res_df = pd.DataFrame(result[1:], columns=result[0])
res_df

,Cities Count
0,1416


## Show top 10 cities ranked by number of businesses listed.

In [21]:
sample_size = 10
query_str = f'''select city as City, count(*) as 'Number_of_Businesses'
                from business 
                group by city
                order by Number_of_Businesses desc
                limit {sample_size}
                '''
result = dml_man.execute_query(query_str)

# i = 0
# for i, row in enumerate(result):
#     print(row)
#     if i == sample_size:
#         break
res_df = pd.DataFrame(result[1:], columns=result[0])
res_df


,City,Number_of_Businesses
0,Philadelphia,14569
1,Tucson,9250
2,Tampa,9050
3,Indianapolis,7540
4,Nashville,6971
5,New Orleans,6209
6,Reno,5935
7,Edmonton,5054
8,Saint Louis,4827
9,Santa Barbara,3829


## Show top 10 cities ranked by number of businesses - that are restaurants - listed.
## Also show distribution in the percentage terms

In [22]:
sample_size = 10 # int or None
query_str = f'''
                with cte AS (
                    select subq.city, count(*) as Number_of_Restaurants
                    from (select * from business
                            where lower(categories) like '%restaurant%'
                            ) as subq
                    group by subq.city
                    order by Number_of_Restaurants desc
                    )
                select city as City, Number_of_Restaurants, 
                    round((100*Number_of_Restaurants / 
                        (select SUM(Number_of_Restaurants) from cte)), 2)
                            as Percentage_of_Total
                from cte
                order by Percentage_of_Total desc
                limit {sample_size}
                ''' 

result = dml_man.execute_query(query_str)
# if result and sample_size:
res_df = pd.DataFrame(result[1:], columns=result[0])
res_df

#     i = 0
#     for i, row in enumerate(result):
#         print(row)
#         if i == sample_size:
#             break


,City,Number_of_Restaurants,Percentage_of_Total
0,Philadelphia,5854,11.0
1,Tampa,2961,5.0
2,Indianapolis,2862,5.0
3,Nashville,2503,4.0
4,Tucson,2470,4.0
5,New Orleans,2260,4.0
6,Edmonton,2166,4.0
7,Saint Louis,1791,3.0
8,Reno,1288,2.0
9,Boise,849,1.0


### Get the ids and names of the top 10 users who have written most number of tips in the year 2015.

In [23]:
# sample_size = 10 # int or None
# query_str = f'''
#                 select t.user_id, u.name, count(*) as number_of_tips
#                 from tip as t  left join user as u
#                 on t.user_id
#                 group by t.user_id
#                 order by number_of_tips desc
#                 limit {sample_size}
#                 ''' 


# result = dml_man.execute_query(query_str)
# res_df = pd.DataFrame(result[1:], columns=result[0])
# res_df



## Find the top 10 businesses with the highest number of reviews, along with their categories

In [24]:
sample_size = 10 # int or None
query_str = f"""
            SELECT b.business_id, b.name, b.review_count, b.categories
            FROM business b
            JOIN (
                SELECT business_id, MAX(review_count) AS max_review_count
                FROM business
                GROUP BY business_id
                ORDER BY max_review_count DESC
                LIMIT {sample_size}
            ) AS top_businesses
            ON b.business_id = top_businesses.business_id
            ORDER BY b.review_count DESC
            """

result = dml_man.execute_query(query_str)
res_df = pd.DataFrame(result[1:], columns=result[0])
res_df



,business_id,name,review_count,categories
0,_ab50qdWOk0DdB6XOrBitw,Acme Oyster House,7568,"Live/Raw Food, Seafood, Restaurants, Cajun/Creole"
1,ac1AeYqs8Z4_e2X5M3if2A,Oceana Grill,7400,"Restaurants, Seafood, Cajun/Creole, Breakfast ..."
2,GXFMD0Z4jEVZBCsbPf4CTQ,Hattie B’s Hot Chicken - Nashville,6093,"American (Traditional), Chicken Shop, Southern..."
3,ytynqOUb3hjKeJfRj5Tshw,Reading Terminal Market,5721,"Candy Stores, Shopping, Department Stores, Fas..."
4,oBNrLz4EDhiscSlbOl8uAw,Ruby Slipper - New Orleans,5193,"Restaurants, American (Traditional), American ..."
5,iSRTaT9WngzB8JJ2YKJUig,Mother's Restaurant,5185,"Cajun/Creole, Restaurants, Event Planning & Se..."
6,VQcCL9PiNL_wkGf-uF3fjg,Royal House,5070,"American (New), Restaurants, Sandwiches, Seafo..."
7,_C7QiQQc47AOEv4PE3Kong,Commander's Palace,4876,"French, Restaurants, Cocktail Bars, Nightlife,..."
8,GBTPC53ZrG1ZBY3DT8Mbcw,Luke,4554,"German, Restaurants, Seafood, Cocktail Bars, F..."
9,6a4gLLFSgr-Q6CZXDLzBGQ,Cochon,4421,"Cajun/Creole, Seafood, Restaurants"


## Find the average number of stars received by businesses in each state; 
## Rank the states by average star rating. We are only interested in top 5 states.

In [25]:

sample_size = 5 # int or None
query_str = f"""
            SELECT state, round(AVG(stars), 2) AS avg_stars
            FROM business
            GROUP BY state
            ORDER BY avg_stars DESC
            limit {sample_size};
            """

result = dml_man.execute_query(query_str)
res_df = pd.DataFrame(result[1:], columns=result[0])
res_df



,state,avg_stars
0,MT,5.0
1,XMS,4.0
2,VT,4.0
3,UT,4.0
4,SD,4.0


In [26]:

sample_size = 5 # int or None
query_str = f"""
            SELECT 
                AVG(u.compliment_hot) / total_compliments AS avg_hot_compliment,
                AVG(u.compliment_more) / total_compliments AS avg_more_compliment,
                AVG(u.compliment_profile) / total_compliments AS avg_profile_compliment,
                AVG(u.compliment_cute) / total_compliments AS avg_cute_compliment,
                AVG(u.compliment_list) / total_compliments AS avg_list_compliment,
                AVG(u.compliment_note) / total_compliments AS avg_note_compliment,
                AVG(u.compliment_plain) / total_compliments AS avg_plain_compliment,
                AVG(u.compliment_cool) / total_compliments AS avg_cool_compliment,
                AVG(u.compliment_funny) / total_compliments AS avg_funny_compliment,
                AVG(u.compliment_writer) / total_compliments AS avg_writer_compliment,
                AVG(u.compliment_photos) / total_compliments AS avg_photos_compliment
            FROM user u
            JOIN review r ON u.user_id = r.user_id
            JOIN business b ON r.business_id = b.business_id
            CROSS JOIN (
                SELECT 
                    SUM(u.compliment_hot + u.compliment_more + u.compliment_profile + u.compliment_cute + 
                        u.compliment_list + u.compliment_note + u.compliment_plain + u.compliment_cool + 
                        u.compliment_funny + u.compliment_writer + u.compliment_photos) AS total_compliments
                FROM user u
                JOIN review r ON u.user_id = r.user_id
                JOIN business b ON r.business_id = b.business_id
                WHERE LOWER(b.categories) LIKE '%restaurant%'
            ) AS total
            WHERE LOWER(b.categories) LIKE '%restaurant%';
        
            """

result = dml_man.execute_query(query_str)
res_df = pd.DataFrame(result[1:], columns=result[0])
res_df



,avg_hot_compliment,avg_more_compliment,avg_profile_compliment,avg_cute_compliment,avg_list_compliment,avg_note_compliment,avg_plain_compliment,avg_cool_compliment,avg_funny_compliment,avg_writer_compliment,avg_photos_compliment
0,2.147250e-08,3.044339e-09,1.939235e-09,1.256848e-09,8.392009e-10,2.039360e-08,5.542669e-08,3.845886e-08,3.845886e-08,1.421344e-08,1.615111e-08
